<a href="https://colab.research.google.com/github/Shreyans06/ML-/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import math
import csv
class Node:
    def __init__(self,attribute):
        self.attribute=attribute
        self.children=[]
        self.answer=""
    def __str__(self):
        return self.attribute

def subtables(data,col,delete):
    dict={}
    items=np.unique(data[:,col])
    count=np.zeros((items.shape[0],1),dtype=np.int32)
    for x in range(items.shape[0]):
        for y in range(data.shape[0]):
            if data[y,col]==items[x]:
                count[x]+=1
    for x in range(items.shape[0]):
        dict[items[x]]=np.empty((int(count[x]),data.shape[1]),dtype="|S32")
        pos=0
        for y in range(data.shape[0]):
            if data[y,col]==items[x]:
                dict[items[x]][pos]=data[y]
                pos+=1
        if delete:
            dict[items[x]]=np.delete(dict[items[x]],col,1)
    return items,dict

def entropy(s):
    items=np.unique(s)
    if items.size==1:
        return 0
    counts=np.zeros((items.shape[0],1))
    sums=0
    for x in range(items.shape[0]):
        counts[x]=sum(s==items[x])/(s.size*1.0)
    for count in counts:
        sums+=-1*count*math.log(count,2)
    return sums

def gain_ratio(data,col):
    items,dict=subtables(data,col,delete=False)
    total_size=data.shape[0]
    entropies=np.zeros((items.shape[0],1))
    intrinsic=np.zeros((items.shape[0],1))
    for x in range(items.shape[0]):
        ratio=dict[items[x]].shape[0]/(total_size*1.0)
        entropies[x]=ratio*entropy(dict[items[x]][:,-1])
        intrinsic[x]=ratio*math.log(ratio,2)
    total_entropy=entropy(data[:,-1])
    iv=-1*sum(intrinsic)
    for x in range(entropies.shape[0]):
        total_entropy-=entropies[x]
    return total_entropy/iv

def create_node(data,metadata):
    if(np.unique(data[:,-1])).shape[0]==1:
        node=Node(" ")
        node.answer=np.unique(data[:,-1])[0]
        return node
    gains=np.zeros((data.shape[1]-1,1))
    for col in range(data.shape[1]-1):
        gains[col]=gain_ratio(data,col)
    split=np.argmax(gains)
    node=Node(metadata[split])
    metadata=np.delete(metadata,split,0)
    items,dict=subtables(data,split,delete=True)
    for x in range(items.shape[0]):
        child=create_node(dict[items[x]],metadata)
        node.children.append((items[x],child))
    return node

def empty(size):
    S=""
    for x in range(size):
        S+=" "
    return S

def print_tree(node,level):
    if node.answer!="":
        print(empty(level),node.answer)
        return
    print(empty(level),node.attribute)
    for value,n in node.children:
        print(empty(level+1),value)
        print_tree(n,level+2)
        
with open('lab3.csv') as csvFile:
    examples=[list(line) for line in csv.reader(csvFile)]
traindata=examples[1:]
metadata=examples[0]

data=np.array(traindata)
node=create_node(data,metadata)
print_tree(node,0)

 outlook
  overcast
   b'yes'
  rain
   wind
    b'strong'
     b'no'
    b'weak'
     b'yes'
  sunny
   humidity
    b'high'
     b'no'
    b'normal'
     b'yes'
